In [1]:
import visualize_data as viz
import regex_def as rd
import re 
import numpy as np
from os import walk
import os

##### Get list of log files

In [2]:
def get_all_files(dirname):
    f = []
    for (dirpath, dirnames, filenames) in walk(dirname):
        f.extend(filenames)
        break
    #print(f)
    return f

#### Get state sequences

In [3]:
def get_states_sequences():
    #get all logs
    logs_all = []
    dirname = r"C:\Users\sab\Downloads\Vulnerablity\SmartDelta\Akka\samples\poc_new\architecture_visualization_tool\dualstackLogs\elevator"
    file_list = get_all_files(dirname)
    for file in file_list:
        state_seq = open(os.path.join(dirname, file), 'r')
        Lines = state_seq.readlines() 
        state_changes_all = []
        for Line in Lines:
            states_trans =[]
            states_trans = Line.split(" ")
            #state_change_in_line = []
            init_state = next_state = ""
            for state in states_trans:
                if(re.match(rd.trace_regex_message, state)):
                    #only select messages that are state changes 
                    m = re.match(rd.state_from, state)
                    if(m is not None):
                        init_state = m.group("state_from")
                    m = re.match(rd.state_to, state)
                    if(m is not None):
                        next_state = m.group("state_to")        
                
                    if(len(init_state) >0 and len(next_state)>0):
                        state_changes_all.append([init_state[:-1], next_state[:-1]])
        #print(state_changes_all)
        logs_all.append(state_changes_all)
    #print(logs_all)
    return (logs_all)

#get_states_sequences()


#### create matrix of values from sequence of states

In [4]:
#This give an array of matrices. 
#Each matrix representing state transition counts in logs
#so for n logs, each having m entries, this will produce a (n,m,m) dimensional array
def get_matrix():
      get_all_states = rd.states
      all_state_sequences = get_states_sequences()
      all_state_seq_mat = []
      for state_sequences in all_state_sequences:
            state_trans_matrix = []
            for source_state in get_all_states:
                  state_trans_count = []
                  for dest_state in get_all_states:
                        transition = [source_state, dest_state]
                        count= state_sequences.count(transition)
                        state_trans_count.append(count)         
                        #print(transition, count)
                  state_trans_matrix.append(state_trans_count)            
            all_state_seq_mat.append(state_trans_matrix)
      print(all_state_seq_mat)
      print("dim:", np.array(all_state_seq_mat).shape)
      return  all_state_seq_mat

#### get l2 distance between the matrices

In [5]:
all_state_seq_mat = get_matrix()

def get_similarity_distance():
    overall_dist = []
    #now file l2 distance between all matrices.
    for state_matrix in all_state_seq_mat:    
        l2_distance_row = []    #contains l2 distance between a single matrix and all other matrices
        for state_matrix_2 in all_state_seq_mat:
            l2= np.linalg.norm(np.array(state_matrix)-np.array(state_matrix_2))
            l2_distance_row.append(l2)
        overall_dist.append(l2_distance_row)
    print(overall_dist)
    return(overall_dist)


[[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 1, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0,

#### Visualize

In [6]:
file_list = get_all_files(r"C:\Users\sab\Downloads\Vulnerablity\SmartDelta\Akka\samples\poc_new\architecture_visualization_tool\dualstackLogs\elevator")
distance_matrix = get_similarity_distance()
viz.plot_data_points_logs(distance_matrix, file_list)

[[0.0, 1.7320508075688772, 5.744562646538029, 0.0, 4.358898943540674, 4.358898943540674, 4.123105625617661, 4.123105625617661], [1.7320508075688772, 0.0, 5.477225575051661, 1.7320508075688772, 4.0, 4.0, 4.242640687119285, 4.47213595499958], [5.744562646538029, 5.477225575051661, 0.0, 5.744562646538029, 2.449489742783178, 2.449489742783178, 2.8284271247461903, 2.8284271247461903], [0.0, 1.7320508075688772, 5.744562646538029, 0.0, 4.358898943540674, 4.358898943540674, 4.123105625617661, 4.123105625617661], [4.358898943540674, 4.0, 2.449489742783178, 4.358898943540674, 0.0, 0.0, 1.4142135623730951, 2.0], [4.358898943540674, 4.0, 2.449489742783178, 4.358898943540674, 0.0, 0.0, 1.4142135623730951, 2.0], [4.123105625617661, 4.242640687119285, 2.8284271247461903, 4.123105625617661, 1.4142135623730951, 1.4142135623730951, 0.0, 1.4142135623730951], [4.123105625617661, 4.47213595499958, 2.8284271247461903, 4.123105625617661, 2.0, 2.0, 1.4142135623730951, 0.0]]


,elevator2.log,elevator3.log,elevator4.log,elevator5.log,elevator6.log,elevator7.log,elevator8.log,elevator9.log
elevator2.log,0.000,1.732,5.745,0.000,4.359,4.359,4.123,4.123
elevator3.log,1.732,0.000,5.477,1.732,4.000,4.000,4.243,4.472
elevator4.log,5.745,5.477,0.000,5.745,2.449,2.449,2.828,2.828
elevator5.log,0.000,1.732,5.745,0.000,4.359,4.359,4.123,4.123
elevator6.log,4.359,4.000,2.449,4.359,0.000,0.000,1.414,2.000
elevator7.log,4.359,4.000,2.449,4.359,0.000,0.000,1.414,2.000
elevator8.log,4.123,4.243,2.828,4.123,1.414,1.414,0.000,1.414
elevator9.log,4.123,4.472,2.828,4.123,2.000,2.000,1.414,0.000


In [7]:
import json
b = rd.states
res = []
for data in distance_matrix:
    res.append(dict((zip(b,data))))
    #print(res)
f= json.dumps(res)
print(f)

[{"user1.Initial": 0.0, "user1.ArriveFloorIN": 1.7320508075688772, "user1.waiting": 5.744562646538029, "user1.elevator": 0.0, "user1.elevator.Initial": 4.358898943540674, "user1.elevator.callreceived": 4.358898943540674, "user1.elevator.decision": 4.123105625617661, "user1.elevator.decision.Initial": 4.123105625617661}, {"user1.Initial": 1.7320508075688772, "user1.ArriveFloorIN": 0.0, "user1.waiting": 5.477225575051661, "user1.elevator": 1.7320508075688772, "user1.elevator.Initial": 4.0, "user1.elevator.callreceived": 4.0, "user1.elevator.decision": 4.242640687119285, "user1.elevator.decision.Initial": 4.47213595499958}, {"user1.Initial": 5.744562646538029, "user1.ArriveFloorIN": 5.477225575051661, "user1.waiting": 0.0, "user1.elevator": 5.744562646538029, "user1.elevator.Initial": 2.449489742783178, "user1.elevator.callreceived": 2.449489742783178, "user1.elevator.decision": 2.8284271247461903, "user1.elevator.decision.Initial": 2.8284271247461903}, {"user1.Initial": 0.0, "user1.Arriv